This script is used to convert toggl time entries to 7pace

# Requirements
- Create the .env file starting from the .env.template file
- Toggl entries should be the result of the work item copy button
- Configurate the #Constants block

Set variables from environment variables

In [1]:
import os
import sys
from dotenv import load_dotenv

load_dotenv(override=True)

# Constants

TOGGL_API_KEY = os.getenv('TOGGL_API_KEY')
TOGGL_WORKSPACE_ID = os.getenv('TOGGL_WORKSPACE_ID')
TOGGL_PROJECT_ID = os.getenv('TOGGL_PROJECT_ID')
SEVENPACE_API_KEY = os.getenv('SEVENPACE_API_KEY')
SEVENPACE_API_VERSION = os.getenv('SEVENPACE_API_VERSION')
DEVOPS_PERSONAL_ACCESS_TOKEN = os.getenv('DEVOPS_PERSONAL_ACCESS_TOKEN')

# If True, will only consider work items that were updated in the last week (vs current week)
LAST_WEEK = eval(os.getenv('LAST_WEEK'))

# Will be considered a TFS work item (not Azure Devops) if the work item id is greater than this number
TFS_7PACE_WORK_ITEM_THRESHOLD_START = int(os.getenv('TFS_7PACE_WORK_ITEM_THRESHOLD_START'))


In [2]:
import requests

# HTTP GET request to Toggl API


def TogglHttpGet(url, params):
    response = requests.get(url, auth=(
        TOGGL_API_KEY, 'api_token'), params=params)
    return response.json()


In [3]:
# Load the start and end dates of the current day week
from datetime import datetime, timedelta
day = datetime.now().strftime('%d/%b/%Y')
dt = datetime.strptime(day, '%d/%b/%Y')
start_day = (dt - timedelta(days=dt.weekday()))
if LAST_WEEK:
    start_day = start_day - timedelta(days=7)
end_day = start_day + timedelta(days=6)
print("Start: " + start_day.strftime('%Y-%m-%d'))
print("End: " + end_day.strftime('%Y-%m-%d'))


Start: 2023-12-04
End: 2023-12-10


In [4]:
import json

# Load toggl time entries for the specified project
TogglFetchEntriesUrl = 'https://api.track.toggl.com/api/v9/me/time_entries'
TimeEntries = TogglHttpGet(TogglFetchEntriesUrl, {
    'start_date': start_day.strftime('%Y-%m-%d'),
    'end_date': end_day.strftime('%Y-%m-%d')
})
# Filter time entries for the specified project
TimeEntries = [entry for entry in TimeEntries if entry['pid']
               == int(TOGGL_PROJECT_ID)]
print(json.dumps(TimeEntries, indent=4))


[
    {
        "id": 3241189799,
        "workspace_id": 3602019,
        "project_id": 190545217,
        "task_id": null,
        "billable": false,
        "start": "2023-12-08T23:07:21+00:00",
        "stop": "2023-12-08T23:30:51Z",
        "duration": 1410,
        "description": "Feature 80836: Login refactoring",
        "tags": [],
        "tag_ids": [],
        "duronly": true,
        "at": "2023-12-08T23:30:51+00:00",
        "server_deleted_at": null,
        "user_id": 5025026,
        "uid": 5025026,
        "wid": 3602019,
        "pid": 190545217
    },
    {
        "id": 3241131711,
        "workspace_id": 3602019,
        "project_id": 190545217,
        "task_id": null,
        "billable": false,
        "start": "2023-12-08T21:33:48+00:00",
        "stop": "2023-12-08T23:07:21Z",
        "duration": 5613,
        "description": "SwatTask 3427611: [Syracuse City School District] Web app Role gives warning \" failed to start agent\" if add failover server",
        

In [5]:
# Clear 7pace time entries for this week
import requests

url = f"https://geneteccentral.timehub.7pace.com/api/rest/workLogs/?api-version={SEVENPACE_API_VERSION}"
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {SEVENPACE_API_KEY}'
}
response = requests.request("GET", url, headers=headers)
currentWeekWorklogs = []
for worklog in response.json()['data']:
  if worklog['timestamp'] >= start_day.strftime('%Y-%m-%d') and worklog['timestamp'] <= end_day.strftime('%Y-%m-%d'):
    currentWeekWorklogs.append(worklog)

if (len(currentWeekWorklogs) == 0):
  print("No entries already there in 7pace for this week")
else:
  print(json.dumps(
      list(map(lambda x: x['comment'], currentWeekWorklogs)), indent=2))

  inputResp = input(
      f"Deleting {str(len(currentWeekWorklogs))} entries. Continue ? (y/n)")
  if inputResp.lower() == 'n':
      exit

  error = False

  for worklog in currentWeekWorklogs:
    url = f"https://geneteccentral.timehub.7pace.com/api/rest/workLogs/{worklog['id']}?api-version={SEVENPACE_API_VERSION}"
    response = requests.request("DELETE", url, headers=headers)
  if not (response.status_code == 204 or response.status_code == 200):
    print("Error deleting worklog " + worklog['id'])
    print(response.text)
    error = True

  if not error:
    print(f"{str(len(currentWeekWorklogs))} worklogs deleted successfully")


[
  "Feature 80836: Login refactoring",
  "Feature 80836: Login refactoring",
  "Bug 146087: Privileges - With some privileges disabled, you can still see the task/subtask and interact with entities.",
  "Daily",
  "Technical Debt 144333: Create the default task user option",
  "Technical Debt 145882: WebApp - Access Rules - Hide sub task behind a feature flag.",
  "User Story 145268: Support privacy protected streams in the webapp",
  "PR 145947: NavWithProfile - Add Setting and Send Feedback to nav bar",
  "Bug 144728: Access Control - Credentials - Save always fail on a certain credential",
  "Bug 145398: WebApp - Refreshing the page takes a long time and then logs you out",
  "Bug 145398: WebApp - Refreshing the page takes a long time and then logs you out",
  "Training: Starbuck Business Accumen",
  "Bug 145398: WebApp - Refreshing the page takes a long time and then logs you out",
  "Daily",
  "Feature 80836: Login refactoring",
  "Daily",
  "Technical Debt 144908: WebApp - Add e

In [6]:
# Fetch 7pace user id
import requests

url = f"https://geneteccentral.timehub.7pace.com/api/rest/me?api-version={SEVENPACE_API_VERSION}"

payload = {}
headers = {
    'Authorization': 'Bearer ' + SEVENPACE_API_KEY
}
response = requests.request("GET", url, headers=headers, data=payload)
if not response.ok:
    print("Error fetching user id", response.text)
else:
  USER_ID = response.json()['data']['user']['id']
  print("USER_ID=" + USER_ID)


USER_ID=c4da23b7-8578-4f8b-994a-a1953d6b5f6a


In [7]:
# Fetch 7pace activities
import requests
import json
url = f"https://geneteccentral.timehub.7pace.com/api/rest/activityTypes?api-version={SEVENPACE_API_VERSION}"
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {SEVENPACE_API_KEY}'
}
resp = requests.request("GET", url, headers=headers, data=payload)

# Retun a name/id map of activities
activities = {}
for activity in resp.json()['data']['activityTypes']:
  activities[activity['id']] = activity['name']
print(json.dumps(activities, indent=2))


{
  "00000000-0000-0000-0000-000000000000": "[Not Set]",
  "140dc2a9-03a0-4431-9f6b-0120f2c1b82f": "Bug Fixing",
  "baebe11d-75e5-4bed-8644-d0ce8731a1b4": "Business development",
  "cd0690af-8e15-4616-a9e3-f3bcfa0f3d5f": "Certification",
  "d0ae0b59-b071-409b-80f9-c9c2626e08a1": "Customer issues",
  "6a79b89e-4c6d-4866-b07b-428b65f1b1d5": "Customer Support and Services",
  "430a57f9-9da2-41e9-97e8-34abdf3e70b2": "Development Infrastructure",
  "9cd0cb39-180d-4b19-9265-83f9753ff76c": "Feature Development",
  "191afffa-22b6-42fc-b93d-36426ae39a6c": "Initial Onboarding",
  "ebccdf5e-93ce-4c94-8c31-c4d57712a28b": "People Centric Activities",
  "2a032e88-7d97-44d3-b28e-ce89b4277017": "Professional Development",
  "ccdf4989-6e17-450c-8c5b-9cfd8496de34": "Regression",
  "bc23a96d-f6c5-44fe-be60-337af432d71b": "Technical Planning",
  "4203e589-2db3-4726-8cb5-f28632ae0838": "Work Absence"
}


In [8]:
# Convert Toggl time entries to 7pace format
from datetime import datetime, timezone, timedelta
import re

SevenPaceTimeEntries = []
for entry in TimeEntries:
    start = entry['start']
    duration = entry['duration']
    if(duration < 0):
      print("Ignoring entry with negative duration (pending timer)")
      continue
    notes = entry['description']
    workItemId = None
    numbers = re.findall(r'\d+:?', notes)
    if numbers:
        workItemId = int(numbers[0].rstrip(':'))

    # Default for Internal Operations
    activityTypeId = "ebccdf5e-93ce-4c94-8c31-c4d57712a28b"
    if ('technical debt' in notes.lower() or 'bug' in notes.lower()):
      activityTypeId = "140dc2a9-03a0-4431-9f6b-0120f2c1b82f"  # Bug
    elif ('feature' in notes.lower() or 'user story' in notes.lower() or 'pr' in notes.lower()):
      activityTypeId = "9cd0cb39-180d-4b19-9265-83f9753ff76c"  # Feature
    elif ('swattask' in notes.lower() or 'swat' in notes.lower()):
      activityTypeId = "d0ae0b59-b071-409b-80f9-c9c2626e08a1"  # Customer issues
    elif ('professional development' in notes.lower() or 'formation' in notes.lower()):
      activityTypeId = "2a032e88-7d97-44d3-b28e-ce89b4277017"  # Professional development

    SevenPaceTimeEntries.append({
        'timeStamp': start,
        'length': duration,
        'lengthFriendly': "{:0>8}".format(str(timedelta(seconds=duration))),
        'comment': notes,
        "workItemId": workItemId,
        'userId': USER_ID,
        'activityTypeId': activityTypeId,
        "activityFriendly": activities[activityTypeId]
    })

print(json.dumps(SevenPaceTimeEntries, indent=2))


[
  {
    "timeStamp": "2023-12-08T23:07:21+00:00",
    "length": 1410,
    "lengthFriendly": "00:23:30",
    "comment": "Feature 80836: Login refactoring",
    "workItemId": 80836,
    "userId": "c4da23b7-8578-4f8b-994a-a1953d6b5f6a",
    "activityTypeId": "9cd0cb39-180d-4b19-9265-83f9753ff76c",
    "activityFriendly": "Feature Development"
  },
  {
    "timeStamp": "2023-12-08T21:33:48+00:00",
    "length": 5613,
    "lengthFriendly": "01:33:33",
    "comment": "SwatTask 3427611: [Syracuse City School District] Web app Role gives warning \" failed to start agent\" if add failover server",
    "workItemId": 3427611,
    "userId": "c4da23b7-8578-4f8b-994a-a1953d6b5f6a",
    "activityTypeId": "d0ae0b59-b071-409b-80f9-c9c2626e08a1",
    "activityFriendly": "Customer issues"
  },
  {
    "timeStamp": "2023-12-08T17:16:13+00:00",
    "length": 15455,
    "lengthFriendly": "04:17:35",
    "comment": "Feature 80836: Login refactoring",
    "workItemId": 80836,
    "userId": "c4da23b7-8578-4f

In [9]:
# Publish worklogs to 7pace

import requests
import json


url = f"https://geneteccentral.timehub.7pace.com/api/rest/workLogs?api-version={SEVENPACE_API_VERSION}"
error = False
tfsEntries = []
devopsEntries = []
for entry in SevenPaceTimeEntries:
  if entry['workItemId'] != None and int(entry['workItemId']) > TFS_7PACE_WORK_ITEM_THRESHOLD_START:
    tfsEntries.append(entry)
  else:
    devopsEntries.append(entry)

    
inputResp = input(f"{str(len(devopsEntries))} entries to add. Continue ? (y/n)")
if inputResp.lower() == 'n':
  exit

for entry in devopsEntries:
  payload = json.dumps(entry)
  headers = {
      'Content-Type': 'application/json',
      'Authorization': f'Bearer {SEVENPACE_API_KEY}'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  print(json.dumps(response.json(), indent=2))
  if (response.status_code != 200):
    error = True

if (not error):
  print("Succeeded")



{
  "data": {
    "timestamp": "2023-12-08T23:07:21Z",
    "length": 1410,
    "billableLength": null,
    "workItemId": 80836,
    "comment": "Feature 80836: Login refactoring",
    "user": {
      "uniqueName": "cnoll@GENETEC.COM",
      "displayName": null,
      "vstsId": "b0010469-3e04-6f0f-860c-9ba717df47db",
      "vstsCollectionId": "b0010469-3e04-6f0f-860c-9ba717df47db",
      "vstsCollectionId2": "b0010469-3e04-6f0f-860c-9ba717df47db",
      "email": "cnoll@GENETEC.COM",
      "name": "Corantin Noll",
      "id": "c4da23b7-8578-4f8b-994a-a1953d6b5f6a"
    },
    "addedByUser": {
      "uniqueName": "cnoll@GENETEC.COM",
      "displayName": null,
      "vstsId": "b0010469-3e04-6f0f-860c-9ba717df47db",
      "vstsCollectionId": "b0010469-3e04-6f0f-860c-9ba717df47db",
      "vstsCollectionId2": "b0010469-3e04-6f0f-860c-9ba717df47db",
      "email": "cnoll@GENETEC.COM",
      "name": "Corantin Noll",
      "id": "c4da23b7-8578-4f8b-994a-a1953d6b5f6a"
    },
    "editedByUser": {


In [10]:
# Print TFS entries
print("TFS entries: " + str(len(tfsEntries)) + " entries")
if(tfsEntries.__len__() > 0):
  uniqueEntries = []
  for entry in tfsEntries:
    existingEntry = None
    for e in uniqueEntries:
      if e['workItemId'] == entry['workItemId']:
        existingEntry = e
        break
    if not existingEntry:
      uniqueEntries.append(entry)
    else:
      existingEntry['length'] += entry['length']
      existingEntry['lengthFriendly'] = "{:0>8}".format(str(timedelta(seconds=existingEntry['length'])))
  print("TFS entries: ")
  print(json.dumps(uniqueEntries, indent=2))

TFS entries: 7 entries
TFS entries: 
[
  {
    "timeStamp": "2023-12-08T21:33:48+00:00",
    "length": 5613,
    "lengthFriendly": "01:33:33",
    "comment": "SwatTask 3427611: [Syracuse City School District] Web app Role gives warning \" failed to start agent\" if add failover server",
    "workItemId": 3427611,
    "userId": "c4da23b7-8578-4f8b-994a-a1953d6b5f6a",
    "activityTypeId": "d0ae0b59-b071-409b-80f9-c9c2626e08a1",
    "activityFriendly": "Customer issues"
  },
  {
    "timeStamp": "2023-12-08T15:35:09+00:00",
    "length": 26960,
    "lengthFriendly": "07:29:20",
    "comment": "User Story 3676090: Replace Mobius by NYPD logo",
    "workItemId": 3676090,
    "userId": "c4da23b7-8578-4f8b-994a-a1953d6b5f6a",
    "activityTypeId": "9cd0cb39-180d-4b19-9265-83f9753ff76c",
    "activityFriendly": "Feature Development"
  },
  {
    "timeStamp": "2023-12-05T15:15:53+00:00",
    "length": 3543,
    "lengthFriendly": "00:59:03",
    "comment": "EPM Feature 3674768: NYPD - Remove Ge